<a href="https://colab.research.google.com/github/akwesi-duodu/GO2COD_2024_710/blob/main/hdrmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout

In [44]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [45]:
def plot_input_img(i):
    plt.imshow(X_train[i], cmap='binary')
    plt.title(y_train[i])
    plt.show()


obtain data and preprocess

In [46]:
# Pre Process the images

# Normalizing the image to [0,1] range
X_train = X_train.astype(np.float32)/255
X_test = X_test.astype(np.float32)/255

# Reshape / expand the dimentions of images to (28, 28, 1)
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

# convert classes to one hot vectors
# Use tf.keras.utils.to_categorical instead of keras.utils.np_utils.to_categorical
from tensorflow.keras.utils import to_categorical  # Import to_categorical
y_train = to_categorical(y_train, num_classes=10) # Assuming 10 classes for MNIST
y_test = to_categorical(y_test, num_classes=10)

In [47]:
#the model is defined and compiled only once.
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape = (28, 28, 1), activation= 'relu'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64, (3,3), activation= 'relu'))
model.add(MaxPool2D((2,2)))
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss= keras.losses.categorical_crossentropy, metrics=['accuracy'])

# Callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint

#Earlystopping
# monitor should be 'val_accuracy' not 'val_acc'
es = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=4, verbose=1, mode='max')

# Model check point
mc = ModelCheckpoint("/content/bestmodel.keras", monitor="val_accuracy", verbose=1, save_best_only=True)

cb = [es, mc]

# Train the model once
his = model.fit(X_train, y_train, epochs=5, validation_split=0.3, callbacks=cb)

Epoch 1/5
1312/1313 [============================>.] - ETA: 0s - loss: 0.2186 - accuracy: 0.9335
Epoch 1: val_accuracy improved from -inf to 0.97578, saving model to /content/bestmodel.keras
1313/1313 [==============================] - 28s 21ms/step - loss: 0.2185 - accuracy: 0.9335 - val_loss: 0.0794 - val_accuracy: 0.9758
Epoch 2/5
1308/1313 [============================>.] - ETA: 0s - loss: 0.0749 - accuracy: 0.9772
Epoch 2: val_accuracy improved from 0.97578 to 0.98211, saving model to /content/bestmodel.keras
1313/1313 [==============================] - 26s 20ms/step - loss: 0.0748 - accuracy: 0.9772 - val_loss: 0.0586 - val_accuracy: 0.9821
Epoch 3/5
1309/1313 [============================>.] - ETA: 0s - loss: 0.0546 - accuracy: 0.9830
Epoch 3: val_accuracy improved from 0.98211 to 0.98433, saving model to /content/bestmodel.keras
1313/1313 [==============================] - 26s 20ms/step - loss: 0.0547 - accuracy: 0.9830 - val_loss: 0.0517 - val_accuracy: 0.9843
Epoch 4/5
1310/1

In [48]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 13, 13, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 5, 5, 64)          0         
 ng2D)                                                           
                                                                 
 flatten_5 (Flatten)         (None, 1600)              0         
                                                                 
 dropout_5 (Dropout)         (None, 1600)             

model training

In [49]:
from google.colab import files

uploaded = files.upload("/Users/akwesi.duodu/Downloads/bestmodel.keras")


Saving bestmodel.keras to /Users/akwesi.duodu/Downloads/bestmodel.keras/bestmodel (1).keras


In [50]:
score = model.evaluate(X_test, y_test)

print(f"The model Accuracy is {score[1]}")

313/313 [==============================] - 2s 6ms/step - loss: 0.0345 - accuracy: 0.9892
The model Accuracy is 0.9891999959945679
